# 🎫 5. 정수 인코딩 (Integer Encoding): 단어에 번호표 붙이기

> **"컴퓨터야, '사과'는 1번, '바나나'는 2번이야!"**

컴퓨터는 글자를 싫어해. 100% 숫자로 바꿔줘야 계산을 하지.
그래서 모든 단어에게 **고유한 ID 번호**를 부여하는 작업이야.

### 방식
- 자주 나오는 단어일수록 앞번호(1번, 2번...)를 주는 게 국룰이야. (효율성 때문!)

---

## 1. 수동으로 만들어보기 (원리 이해) 🛠️

패키지 쓰기 전에 직접 구현해보면서 원리를 깨우쳐보자.

In [2]:
from nltk.tokenize import word_tokenize
from collections import Counter

sentences = [
    "I love my dog",
    "I love my cat",
    "You love my dog",
    "Do you think my dog is amazing"
]

# 1. 단어들 싹 다 모으기
vocab = []
for sent in sentences:
    vocab.extend(word_tokenize(sent.lower()))

# 2. 빈도수 세기 & 많이 나온 순서 정렬
word_counts = Counter(vocab)
sorted_vocab = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

print("단어 빈도:", sorted_vocab)

단어 빈도: [('my', 4), ('love', 3), ('dog', 3), ('i', 2), ('you', 2), ('cat', 1), ('do', 1), ('think', 1), ('is', 1), ('amazing', 1)]


In [3]:
# 3. 번호표 부여 (1번부터 시작)
word_to_index = {}
i = 0

for word, count in sorted_vocab:
    i += 1
    word_to_index[word] = i

print("단어 사전:", word_to_index)

단어 사전: {'my': 1, 'love': 2, 'dog': 3, 'i': 4, 'you': 5, 'cat': 6, 'do': 7, 'think': 8, 'is': 9, 'amazing': 10}


In [4]:
# 4. 문장을 숫자로 변환!
encoded_sentences = []
for sent in sentences:
    temp = []
    for word in word_tokenize(sent.lower()):
        temp.append(word_to_index[word])
    encoded_sentences.append(temp)

print("변환 결과:", encoded_sentences)

변환 결과: [[4, 2, 1, 3], [4, 2, 1, 6], [5, 2, 1, 3], [7, 5, 8, 1, 3, 9, 10]]


---

## 2. Keras로 한 방에 하기 (feat. 고성능 로봇) 🤖🚀

방금 우리가 **'토큰화 → 정렬 → 인덱싱'** 하며 고생했던 10줄짜리 코드를,
Keras는 **딱 2~3줄**로 끝내버려. 실무에서는 무조건 이걸 쓴다고 보면 돼.

### 2-1. 기본 사용법
Keras의 `Tokenizer`는 우리가 만든 수동 코드와 똑같은 원리로 작동하지만, 훨씬 똑똑해.

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 1. 로봇(Tokenizer) 소환!
tokenizer = Tokenizer()

# 2. 로봇에게 단어 가르치기 (fit_on_texts)
# - 이 한 줄로 '토큰화 + 빈도수 계산 + 정렬 + 인덱스 부여'가 다 끝남!
tokenizer.fit_on_texts(sentences)

print("단어 사전(word_index):", tokenizer.word_index)

### 💡 결과 해석
- `fit_on_texts`를 하면 내부적으로 빈도수를 세서 **가장 많이 나온 단어가 1번**이 돼.
- `word_index`를 찍어보면 우리가 아까 힘들게 만든 `sorted` 결과와 똑같지?

In [6]:
# 3. 문장을 숫자로 변환하기 (texts_to_sequences)
# - 이제 배운 걸 바탕으로 문장을 숫자로 바꿔줘!
encoded = tokenizer.texts_to_sequences(sentences)

print("변환 결과:", encoded)

### 2-2. Keras가 더 좋은 점 (고급 기능) 💎

우리가 짠 코드에는 큰 문제가 하나 있어. **"모르는 단어(Out-Of-Vocabulary)"**가 나오면 에러가 나거나 무시해버린다는 거야.
하지만 Keras는 **`oov_token`** 하나만 설정하면, 모르는 단어도 "모름"이라는 숫자로 바꿔줘!

> **예시**: "I love **zebra**" (zebra는 배운 적 없음)
> - 일반: `[4, 2]` ('zebra' 증발)
> - Keras(OOV 설정 시): `[4, 2, 1]` ('zebra'를 'OOV'에 해당하는 1번으로 변환)

In [7]:
# OOV(Out-Of-Vocabulary) 설정하기
# "모르는 단어가 나오면 'OOV'라는 이름으로 처리하고 번호(1번)를 줘라!"
tokenizer_new = Tokenizer(oov_token='OOV') 
tokenizer_new.fit_on_texts(sentences)

# 'word_index'에 자동으로 'OOV'가 추가됨 (보통 1번)
print("OOV 포함 단어장:", tokenizer_new.word_index)

# 안 배운 단어('zebra')가 있는 문장 변환
new_sentence = ["I love zebra"]
print("변환 결과:", tokenizer_new.texts_to_sequences(new_sentence))

## 🎓 마무리 퀴즈

**Q. 정수 인코딩에서 보통 '가장 많이 나온 단어'에게 부여하는 번호는?**
1. 1번 (앞번호)
2. 10000번 (뒷번호)
3. 랜덤

<details>
<summary>정답 확인</summary>

**1번** (자주 쓰는 건 빨리 찾게 앞번호!)

</details>